# Inventory optimisation algorithm

Balancing stock levels and cost means minimising holding costs (costs of keeping inventory), ordering costs (costs of replenishing stock), and stockout costs (costs of running out of stock). 

## 1. Importing data

In [36]:
import pandas as pd
import numpy as np

file_path = '/Users/yongjun/Documents/School/Y3S1/DSA3101 - Data Science in Practice/Project/cleaned_data.csv'
data = pd.read_csv(file_path)
print(data.head())

   Unnamed: 0  Order Date  Purchase Price Per Unit  Quantity  \
0           1  2018-12-04                     7.98         1   
1           2  2018-12-22                    13.99         1   
2           3  2018-12-24                     8.99         1   
3           4  2018-12-25                    10.45         1   
4           5  2018-12-25                    10.00         1   

  Shipping Address State                                              Title  \
0                     NJ  SanDisk Ultra 16GB Class 10 SDHC UHS-I Memory ...   
1                     NJ  Betron BS10 Earphones Wired Headphones in Ear ...   
2                     NJ                                                NaN   
3                     NJ  Perfecto Stainless Steel Shaving Bowl. Durable...   
4                     NJ                      Proraso Shaving Cream for Men   

  ASIN/ISBN (Product Code)  Survey ResponseID                Category  
0               B0143RTB1E  R_01vNIayewjIIKMF             Electronic

## 2. Setting Parameters
Several key parameters are set for the inventory optimisation:

`ordering_cost`: Fixed cost per order.

`holding_cost_rate`: Percentage of the purchase price representing the annual holding cost.

`service_level_z`: Z-score corresponding to the desired service level (95% in this case).

In [37]:
ordering_cost = 50  # Assuming fixed cost per order
holding_cost_rate = 0.20  # 20% of purchase price as holding cost per year
service_level_z = 1.65  # Z-score for 95% service level

### Ordering Cost:

$50 per order: This is a fixed cost that accounts for administrative, transportation, and handling costs associated with placing each order. The value was set to a moderate level to reflect a reasonable cost in standard e-commerce and supply chain operations. Too high an ordering cost would increase the EOQ excessively, while too low would not accurately capture the overhead.

### Holding Cost Rate:

20% of the purchase price: This rate represents the annual cost of holding inventory, which includes warehousing, insurance, obsolescence, and opportunity cost. A holding cost rate of 20% is typical in many industries and reflects a balance between the cost of capital and the expense of maintaining inventory.

### Service Level Z-score:

Z = 1.65 (95% service level): A 95% service level means there is a 95% probability that demand will not exceed inventory on hand during the lead time. This level was chosen to strike a balance between minimising stockouts and avoiding excessive safety stock.

## 3. Preprocessing Data

The data is preprocessed to compute the monthly and yearly demand for each product. This includes extracting the month and year from the order dates and grouping the data by product titles to sum the quantity sold.

In [38]:
# Preprocessing the data to calculate demand per product
data['Order Date'] = pd.to_datetime(data['Order Date'])
data['Month'] = data['Order Date'].dt.to_period('M')

# Calculate total demand per month for each product
monthly_demand = data.groupby(['Title', 'Month'])['Quantity'].sum().reset_index()

# Calculate total demand per year for each product
data['Year'] = data['Order Date'].dt.year
yearly_demand = data.groupby(['Title', 'Year'])['Quantity'].sum().reset_index()
print(yearly_demand)

                                                     Title  Year  Quantity
0                                        ! Red/Black Swirl  2022         1
1        ![CDATA[Best 1" Ultra-Thin Permanent Double-Si...  2020         1
2        " Ampela Sketching Drawing Pencils for Artists...  2018         1
3        " OCCASIONS " 40 Mugs Pack, Heavyweight Dispos...  2020         2
4        " OCCASIONS" 240 Plates Pack,(120 Guests) Heav...  2018         1
...                                                    ...   ...       ...
1092530  🚌 KiNSFUN 5" Monster School Bus Die Cast Metal...  2022         1
1092531  🚑 KiNSMART Rescue Team Ambulance 5" Die Cast M...  2022         1
1092532  🥚 Set of 2 Surprise Growing Hatching Rainbow E...  2020         3
1092533  🥚 Set of 2 Surprise Growing Hatching Rainbow E...  2021         3
1092534  🥚 Set of 2 Surprise Growing Hatching Rainbow E...  2022         3

[1092535 rows x 3 columns]


## 4. Product Categories: Lead Times and Shelf Life

Lead times and shelf lives are defined for each product category using dictionaries. This information is crucial for determining when products should be reordered and how long they can be stored.

In [39]:
lead_time_dict = {
    'Electronics': 20,
    'Kitchen & Dining': 15,
    'Beauty & Personal Care': 10,
    'Computer & Accessories': 20,
    'Clothing, Shoes and Jewellery': 15,
    'Health, Household & Personal Care': 10,
    'Tools & Home Improvement': 12,
    'Gift Cards': 1,  # Digital products usually have instant delivery
    'Grocery': 5,
    'Home': 15,
    'Office Products': 10,
    'Video Games': 5,
    'Pet Supplies': 7,
    'Books': 7,
    'Toys and Games': 7,
    'Garden & Outdoor': 15,
    'Industrial & Scientific': 25,
    'Sports and Outdoors': 12,
    'Arts, Crafts & Sewing': 7,
    'Baby': 7,
    'Automotive': 20,
    'Luggage & Travel Gear': 12,
    'Movies & TV': 5,
    'CDs and Vinyl': 7,
    'Software': 3,  # For digital versions
    'Musical Instruments': 25,
    'Amazon Fresh': 2,  # Perishables, very short lead time
    'Luxury Beauty': 7,
    'Miscellaneous': 10,
    'Prime Video': 1,  # Instant delivery for digital goods
    'Watsons': 5  # Health and beauty related
}

shelf_life_dict = {
    'Electronics': 730,  # Around 2 years
    'Kitchen & Dining': 365,  # 1 year
    'Beauty & Personal Care': 180,  # 6 months
    'Computer & Accessories': 730,  # 2 years
    'Clothing, Shoes and Jewellery': 730,  # 2 years
    'Health, Household & Personal Care': 365,  # 1 year
    'Tools & Home Improvement': 365,  # 1 year
    'Gift Cards': 365,  # 1 year (may vary)
    'Grocery': 30,  # Perishables
    'Home': 365,  # 1 year
    'Office Products': 365,  # 1 year
    'Video Games': 730,  # 2 years
    'Pet Supplies': 180,  # 6 months
    'Books': 1095,  # 3 years
    'Toys and Games': 365,  # 1 year
    'Garden & Outdoor': 365,  # 1 year
    'Industrial & Scientific': 730,  # 2 years
    'Sports and Outdoors': 365,  # 1 year
    'Arts, Crafts & Sewing': 365,  # 1 year
    'Baby': 180,  # 6 months
    'Automotive': 730,  # 2 years
    'Luggage & Travel Gear': 730,  # 2 years
    'Movies & TV': 365,  # 1 year
    'CDs and Vinyl': 730,  # 2 years
    'Software': None,  # No shelf life for digital
    'Musical Instruments': 730,  # 2 years
    'Amazon Fresh': 7,  # Very perishable
    'Luxury Beauty': 180,  # 6 months
    'Miscellaneous': 365,  # 1 year
    'Prime Video': None,  # No shelf life for digital
    'Watsons': 180  # Health and beauty related, short shelf life
}

### Lead Time
The choice in values for lead time is heavily influenced by the product category. For example, perishable goods (e.g. `Amazon Fresh`, `Grocery`) have very short lead times (2–7 days) due to their quick turnover and necessity for freshness. Digital products (e.g. `Gift Cards`, `Software`, `Prime Video`) have extremely short lead times (1–3 days or instant) as they do not require physical delivery.

However, for `Electronics` and `Computer & Accessories`, these items generally require longer lead times (20 days) due to complex manufacturing processes, high demand, and the supply chain duration from outsourcing.

A longer lead time increases the need for safety stock to avoid stockouts, while shorter lead times reduce the need for excess inventory and lead to more frequent orders.

### Shelf Life
Shelf life is primarily influenced by the perishability of products.
`Grocery` and `Amazon Fresh` have very short shelf lives (30 days or less) due to the perishability of food and other consumables. `Beauty & Personal Care`, `Pet Supplies`, and `Baby Products` have medium shelf lives (180 days or 6 months), as they can expire or degrade in quality over time. `Software` and `Prime Video` are digital products and have no shelf life, as they don’t physically expire or degrade.

Products with shorter shelf lives require more careful inventory management, as ordering too much leads to spoilage and wastage, while long-shelf-life products allow more flexibility in ordering larger quantities.

## 5. Key Calculations

### Economic Order Quantity (EOQ)
EOQ is calculated using the formula:

$$
EOQ = \sqrt{\frac{2DS}{H}}
$$

Where:
- D = Demand rate (units per period)
- S = Ordering cost per order
- H = Holding cost per unit per period
​

This balances the trade-off between ordering and holding costs.

In [40]:
def calculate_eoq(demand, purchase_price):
    holding_cost = holding_cost_rate * purchase_price
    eoq = np.sqrt((2 * demand * ordering_cost) / holding_cost)
    return eoq

### Safety Stock
Safety stock accounts for demand variability during the lead time. It's calculated using:

$$
SS = Z \times \sigma \times \sqrt{LT}
$$

Where:
- SS = Safety Stock
- Z = Z-score (service factor, representing the desired service level)
- 𝜎 = Standard deviation of demand
- LT = Lead time (in the same time units as demand)

In [41]:
def calculate_safety_stock(demand_std, lead_time, service_level_z):
    safety_stock = service_level_z * demand_std * np.sqrt(lead_time)
    return safety_stock

### Reorder Point

The reorder point is the sum of average demand during the lead time and the safety stock:

$$
ROP = (Avg. \, Monthly \, Demand \times Lead \, Time) + Safety \, Stock
$$

---

### Total Cost

The Total Cost (TC) can be calculated using the following formula:

$$
Total \, Cost = \left( \frac{EOQ}{2} \times Holding \, Cost \right) + \left( \frac{Annual \, Demand}{EOQ} \times Ordering \, Cost \right)
$$

## 6. Processing Products

The script loops through all the products, calculates EOQ, safety stock, reorder points, and total costs for each product, and stores the results in a dataframe.

In [42]:
results = []
products = data['Title']
products_truncated = data['Title'][:50]

try:
    for product in products_truncated:
        product_data = yearly_demand[yearly_demand['Title'] == product]
        
        # Check if product_data is not empty
        if not product_data.empty:
            # Annual demand estimation (average demand for all years)
            annual_demand = np.ceil(product_data['Quantity'].mean())
            
            # Purchase price (assuming it's relatively stable)
            avg_purchase_price = data[data['Title'] == product]['Purchase Price Per Unit'].mean()
            
            # Standard deviation of monthly demand (for safety stock calculation)
            demand_std = monthly_demand[monthly_demand['Title'] == product]['Quantity'].std()
            
            # Get product category (only if it exists)
            product_category_data = data[data['Title'] == product]
            if not product_category_data.empty:
                product_category = product_category_data['Category'].iloc[0]
            else:
                product_category = 'Miscellaneous'  # Default to a safe category
            
            # Get the shelf life and lead time for the product's category
            shelf_life = shelf_life_dict.get(product_category, 365)  # Default to 1 year if category not found
            lead_time_days = lead_time_dict.get(product_category, 10)  # Default to 10 days if category not found
            
            # Calculate maximum demand during shelf life (assuming linear demand throughout the year)
            max_demand_shelf_life = (annual_demand / 365) * shelf_life
            
            # EOQ calculation, capped by maximum demand during shelf life
            eoq = min(calculate_eoq(annual_demand, avg_purchase_price), max_demand_shelf_life)
            
            # Safety stock calculation
            safety_stock = calculate_safety_stock(demand_std, lead_time_days, service_level_z)
            
            # Reorder point (ROP) calculation
            avg_monthly_demand = annual_demand / 12
            reorder_point = (avg_monthly_demand * lead_time_days) + safety_stock
            
            # Total cost (TC) calculation
            holding_cost = (eoq / 2) * (holding_cost_rate * avg_purchase_price)
            ordering_cost_total = (annual_demand / eoq) * ordering_cost
            total_cost = holding_cost + ordering_cost_total
            
            # Round values
            eoq_rounded = np.ceil(eoq)  # Round up EOQ to the next integer
            safety_stock_rounded = np.ceil(safety_stock)  # Round up safety stock to the next integer
            reorder_point_rounded = np.ceil(reorder_point)  # Round up reorder point to the next integer
            total_cost_rounded = round(total_cost, 2)  # Round total cost to 2 decimal places
            
            # Append results with lead time and shelf life
            results.append({
                'Title': product,
                'Category': product_category,
                'Annual Demand': annual_demand,
                'EOQ': eoq_rounded,
                'Safety Stock': safety_stock_rounded,
                'Reorder Point': reorder_point_rounded,
                'Lead Time (days)': lead_time_days,
                'Shelf Life (days)': shelf_life,
                'Total Cost': total_cost_rounded
            })
            
            # Print processed information for debugging
            print(f"Processed product: {product}, Lead Time: {lead_time_days}, EOQ: {eoq_rounded}, Total Cost: {total_cost_rounded}")
        else:
            print(f"No data found for product: {product}")
    
except Exception as e:
    print(f"An error occurred: {e}")

results_df = pd.DataFrame(results)

Processed product: SanDisk Ultra 16GB Class 10 SDHC UHS-I Memory Card up to 80MB/s (SDSDUNC-016G-GN6IN), Lead Time: 20, EOQ: 12.0, Total Cost: 34.41
Processed product: Betron BS10 Earphones Wired Headphones in Ear Noise Isolating Earbuds with Microphone and Volume Control Powerful Bass Driven Sound, 12mm Large Drivers, Ergonomic Design, Lead Time: 20, EOQ: 10.0, Total Cost: 39.58
No data found for product: nan
Processed product: Perfecto Stainless Steel Shaving Bowl. Durable Metal Mug For Shaving Soap & Cream. Perfect Addition To Your Wet Shaving Kit. Double Layer Smooth Shave Unbreakable Mug With Heat Insulation, Lead Time: 15, EOQ: 1.0, Total Cost: 51.04
Processed product: Proraso Shaving Cream for Men, Lead Time: 10, EOQ: 2.0, Total Cost: 102.76
Processed product: Micro USB Cable Android Charger - Syncwire [2-Pack 6.6ft] Super-Durable Nylon Braided Fast Sync&Charging Cord for Samsung Galaxy S7 Edge/S7/S6, HTC, LG, Sony, Xbox One, PS4 - Space Grey, Lead Time: 20, EOQ: 4.0, Total Cost

### 7. Outputting Results

The results are saved as a CSV file for further analysis.

In [43]:
results_df.to_csv('/Users/yongjun/Documents/School/Y3S1/DSA3101 - Data Science in Practice/Project/inventory_optimisation.csv', index=False)
print("Optimisation completed")

Optimisation completed


## Price Elasticity

In [44]:
def calculate_yearly_price_elasticity(data, category, top_n_products=50):
    # filtering data for the specified category and top N products
    category_data = data[data['Category'] == category]
    top_products = category_data['Title'].unique()[:top_n_products]
    category_data = category_data[category_data['Title'].isin(top_products)]

    category_data['Order Date'] = pd.to_datetime(category_data['Order Date'])
    category_data = category_data.set_index('Order Date')

    elasticity_results = []

    # calculating elasticity for each product
    for product in top_products:
        product_data = category_data[category_data['Title'] == product]
        product_data = product_data.dropna(subset=['Purchase Price Per Unit', 'Quantity'])
        
        if product_data.empty:
            continue

        yearly_data = product_data.resample('YE').agg({
            'Purchase Price Per Unit': 'mean',
            'Quantity': 'sum'
        }).dropna()

        if yearly_data.empty:
            continue

        yearly_data['Rolling Price Avg'] = yearly_data['Purchase Price Per Unit'].rolling(window=1).mean()
        yearly_data['Price Change %'] = yearly_data['Rolling Price Avg'].pct_change() * 100
        yearly_data['Quantity Change %'] = yearly_data['Quantity'].pct_change() * 100

        if (yearly_data['Price Change %'] == 0).any():
            elasticity_results.append({'Title': product, 'Elasticity': 'No price change'})
            continue

        yearly_data = yearly_data.dropna(subset=['Price Change %', 'Quantity Change %'])
        
        if yearly_data.empty:
            continue

        yearly_data['Elasticity'] = yearly_data['Quantity Change %'] / (yearly_data['Price Change %'] + 1e-9)
        elasticity_avg = yearly_data['Elasticity'].mean()
        elasticity_results.append({'Title': product, 'Elasticity': elasticity_avg})

    # converting results into DataFrame and calculating average elasticity
    elasticity_df = pd.DataFrame(elasticity_results)
    valid_elasticities = elasticity_df[elasticity_df['Elasticity'] != 'No price change']['Elasticity']
    avg_elasticity = valid_elasticities.mean() if not valid_elasticities.empty else None

    return elasticity_df, avg_elasticity

categories = [
    'Electronics',
    'Computer & Accessories',
    'Kitchen & Dining',
    'Beauty & Personal Care',
    'Clothing, Shoes and Jewellery',
]

results = []

# loop through each category and calculate the price elasticities
for category in categories:
    elasticity_df, avg_elasticity = calculate_yearly_price_elasticity(data, category=category, top_n_products=50)
    results.append({
        'Category': category,
        'Elasticity DataFrame': elasticity_df,
        'Average Elasticity': avg_elasticity
    })
    
for result in results:
    print(f"Results for {result['Category']}:")
    print(result['Elasticity DataFrame'])
    print(f"Average Elasticity of {result['Category']}: {result['Average Elasticity']}")
    print("\n")



Results for Electronics:
                                                Title       Elasticity
0   SanDisk Ultra 16GB Class 10 SDHC UHS-I Memory ...         0.833966
1   Betron BS10 Earphones Wired Headphones in Ear ...        -9.873301
2   UGREEN Tablet Stand Holder Adjustable Portable...              0.0
3   Betron B25 in-Ear Headphones Earphones with Mi...        -4.686905
4   SanDisk 128GB Ultra MicroSDXC UHS-I Memory Car...        10.395783
5   Betron BS10 Earphones Wired in Ear Earbud Head...         1.831667
6   Anker Powerline II USB-C to USB-C 3.1 Gen 2 Ca...         3.143082
7   $15/Month Mint Mobile Phone Plan | 4GB of 5G •...        57.549158
8   SAMSUNG (MB-ME256GA/AM) 256GB 100MB/s (U3) Mic...       -12.880041
9   SAMSUNG (MB-ME128GA/AM) 128GB 100MB/s (U3) Mic...        -3.794488
10    $10 -PlayStation Store Gift Card [Digital Code]  No price change
11  USB Type C Cable, Syncwire [2 Pack, 6Ft] USB 3...        18.653333
12  Amazon Basics Carrying Case for Nintendo Switc..